In [ ]:
#from __future__ import unicode_literals
%reload_ext autoreload
%autoreload 2

import os
from os.path import expanduser
home = expanduser("~")
import sys
sys.path.append(home + '/workspace/networkqit')
import numpy as np
import pandas as pd
import scipy
from scipy.io import loadmat
import networkqit as nq
import networkx as nx
import bct
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pickle
import re
# Additional utility functions
from FrontiersAnalysis import data_grid_plot, analyze_forcellini_data, forcellini_plotting, compute_step, collect_step
import gzip
import seaborn as sns

In [ ]:
def percolation_analysis(W,nsteps=10):
    wmin = W[np.nonzero(W)].min()
    wmax = W[np.nonzero(W)].max()
    allw = np.unique(W.flatten())
    all_comps = []
    all_comps_size = []
    allt = np.linspace(wmin,wmax,nsteps)
    giant_size = []
    for t in allt:
        comps, comp_size = bct.get_components(bct.threshold_absolute(W,t))
        all_comps.append(comps)
        all_comps_size.append(len(comp_size))
        giant_size.append(np.max(comp_size))
    return allt,np.array(all_comps_size), np.array(giant_size)

In [ ]:
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')[0:128,0:128]
allt,comps,giantsize = percolation_analysis(W, 100)

In [ ]:
plt.plot(allt,giantsize)
plt.vlines(x=allt[np.array(giantsize)==638][-1],ymin=0,ymax=638,color='k', alpha=0.6)

In [ ]:
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')
beta_range = np.logspace(-3,3,100)

allt = np.linspace(0.4, 0.9, 12)
fig, ax = plt.subplots(ncols=4,nrows=3,figsize=(16,12))
ax = ax.flatten()

all_sol = []
all_smodel = []
from tqdm import tqdm
for i,t in enumerate(allt):
    M = nq.CWTECM(N=len(W), threshold=t)
    G = bct.threshold_absolute(W,t)
    sol = M.fit(G=G, method='MLE', ftol=1)
    all_sol.append(sol)
    
    Amodel = []
    while len(Amodel) < 5:
        X = np.squeeze(M.sample_adjacency(sol['x'], batch_size=1)[0,:,:])
        if bct.number_of_components(X) == 1:
            Amodel.append(X)
    Sdata = nq.batch_compute_vonneumann_entropy(L=nq.graph_laplacian(G), beta_range=beta_range)
    ax[i].semilogx(beta_range,Sdata,label='data',color='b')
    for j in range(len(Amodel)):
        Smodel = nq.batch_compute_vonneumann_entropy(L=nq.graph_laplacian(np.squeeze(Amodel[j])),beta_range=beta_range)
        all_smodel.append(Smodel)
        ax[i].semilogx(beta_range, Smodel, label='model', color='r',alpha=0.25)
    ax[i].legend(['data','model'])
    ax[i].set_title('threshold=%.3f' % t)
    ax[i].set_xlabel('$\\beta$')
    ax[i].set_ylabel('$S(\\beta)$')    
plt.tight_layout()

In [ ]:
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')
beta_range = np.logspace(-4,4,100)

allt = np.linspace(0.4, 0.9, 11)
fig, ax = plt.subplots(ncols=4,nrows=3,figsize=(16,12))
ax = ax.flatten()

for i,t in enumerate(allt):
    M = nq.CWTECM(N=len(W), threshold=t)
    G = bct.threshold_absolute(W,t)
    sol = M.fit(G=G, method='MLE', ftol=1E-3)
    Amodel = []
    count=0
    while len(Amodel) < 10:
        X = np.squeeze(M.sample_adjacency(sol['x'], batch_size=1)[0,:,:])
        if count>50:
            print('Too many iterations')
        if bct.number_of_components(X) ==  bct.number_of_components(G):
            Amodel.append(X)

    for j in range(len(Amodel)):
        dkl = nq.batch_beta_relative_entropy(Lobs=nq.graph_laplacian(G),Lmodel=nq.graph_laplacian(np.array(Amodel)), beta_range=beta_range)
    
    #ax[i].semilogx(beta_range, dkl, label='dkl')
    ax[i].semilogx(beta_range, dkl/(beta_range*np.sum(G)), label='dkl/beta$W_{tot}$')
    
    #ax[i].legend(['$S(\rho \| \sigma)$','$S(\rho \| \sigma)/\beta W_{tot}$'])
    ax[i].set_title('threshold=%.3f' % t)
    ax[i].set_xlabel('$\\beta$')
    ax[i].set_ylabel('$S(\\beta)$')
    
plt.tight_layout()
plt.savefig('dkl_normalized_crossley_groupaverage_mat_thresholded.pdf',bbox='tight')

In [ ]:
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')[10:30,10:30]
allt = np.linspace(0.4, 0.8, 12)
beta_range=np.logspace(-3,3, 100)

def compute_dkl(W,t,beta_range):
    G = bct.threshold_absolute(W,t)
    M = nq.CWTECM(N=len(W), threshold=G[np.nonzero(G)].min())
    sol = M.fit(G=G,method='MLE',ftol=1E-3)
    Amodel = []
    while len(Amodel) < 10:
        X = np.squeeze(M.sample_adjacency(sol['x'], batch_size=1)[0,:,:])
        if bct.number_of_components(X) ==  bct.number_of_components(G):
            Amodel.append(X)
    dkl = nq.batch_beta_relative_entropy(Lobs=nq.graph_laplacian(G),
                                         Lmodel=nq.graph_laplacian(np.array(Amodel)),
                                         beta_range=beta_range)
    return dkl
df = pd.DataFrame(data=np.array([compute_dkl(W,t,np.logspace(-3,3,100)) for t in allt]),columns=['dkl']).set_index(allt)
#df['dklnorm'] = [df['dkl']/(bct.threshold_absolute(W,t).sum()*beta_range) for t in allt ]

In [ ]:
df